In [2]:
import re
import json
from timeout_decorator import timeout

In [3]:
file_name = "RegexEval_Filtered_Manual"

In [4]:
with open(f"./Intermediate_Dataset/{file_name}.json") as f:
    data = json.load(f)

len(data)

1470

In [5]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        print(e)
        return None

In [11]:
new_data = []
for item in data:
    if item['expression'].strip() == '' or item['description'].strip() == '' or item['matches'].strip() == '' or item['non_matches'].strip() == '':
        continue
    
    is_valid = True
    expression = item['expression'].strip()
    description = item['description'].strip()
    matches = [x.strip() for x in item['matches'].strip().split('|')]
    non_matches = [x.strip() for x in item['non_matches'].strip().split('|')]


    for match in matches:
        try:
            is_match = match_expression(expression, match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == False:
            is_valid = False
            break    
    for non_match in non_matches:
        try:
            is_match = match_expression(expression, non_match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == True:
            is_valid = False
            break

    if is_valid:
        new_data.append(item)
    else:
        print('Invalid: ', expression, description, matches, non_matches)


Invalid:  ^[a-zA-Z]\:\\.*|^\\\\.* Useful for determining whether a string constitutes a valid local or UNC path. ['\\\\mypath\\mypath1\\myfile.aaa', 'C:\\mypath\\mypath\\mypath', '\\\\otherpath\\otherpath', 'D:\\somepath\\somefile.file'] ['http://www.mysite.com', '\\\\\\badpath\\badpath', 'X::\\\\badpath']
Invalid:  (\b(1|2|3|4|5|6|7|8|9)?[0-9]\b) Match the numbers 1 to 99 ['1', '2', '99', '10'] ['100', '200', '500', '0']
Invalid:  ^-?[0-9]\d{0,8}(\.\d{1,4}) Numeric with decimal values. Accepts negative and zeros. Upto 9 digits and upto 4 decimals allowed ['123456789.1234', '-123.123', '0.00', '-123456789.1234'] ['1234567890', '0.12345']
Invalid:  (-?[1-9]*\d*[02468])(?=\D) Match even integers, positive or negative, including zero. ['-100', '-10', '-2', '0', '2', '4', '6', '8', '10', '12', '100', '102'] ['-101', '-99', '-11', '-1', '1', '3', '5', '7', '9', '11', '99', '101']
Invalid:  href(\s*)=(\s*)('|")http:\/\/(.*)(("|')) The pulls the HTML link URLs. ['href="http://yahoo.com"', 'hr

/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_17045/1341296894.py:4: FutureWarning: Possible nested set at position 5
  return re.search(expression, text) is not None
/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_17045/1341296894.py:4: FutureWarning: Possible nested set at position 21
  return re.search(expression, text) is not None
/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_17045/1341296894.py:4: FutureWarning: Possible nested set at position 27
  return re.search(expression, text) is not None


In [12]:
with open(f"./Intermediate_Dataset/{file_name}_Filtered.json", 'w') as f:
    json.dump(new_data, f)